<a href="https://colab.research.google.com/github/megkotch09/BluebonnetMay2023/blob/main/NH_33_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas
!pip install matplotlib
!pip install folium
!pip install requests

import pandas as pd
import geopandas as gpd
import folium
import requests
import os
import numbers
import math

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set your Census Bureau API key
api_key = "8a0b76a97aa387f1d801ca752ec8983574f5c307"

# Set the base URL for the API
base_url = "https://api.census.gov/data"

# Set the variables you want to retrieve
variables = ["B01001_001E", "B02001_002E", "B03002_012E", "B02015_002E", "B02015_003E"]  # Example variables (total population, white population, Hispanic population)

# Set the geographic parameters for New Hampshire
state_fips = "33"  # New Hampshire FIPS code

# Construct the API request URL with parameters
api_url = base_url + "/2021/acs/acs5?get=" + ",".join(variables) + "&for=state:" + state_fips + "&key=" + api_key

# Send the request and get the response
response = requests.get(api_url)

# Parse the response data (assuming it's in JSON format)
data = response.json()

In [ ]:
# Extract the variable names from the first row of the response
headers = ["Total", "White", "Hispanic", "Asian Indian", "Bangladeshi", "State"]

# Extract the data values from the second row of the response
rows = data[1:]

# Map the variable names to their corresponding values
df = pd.DataFrame(rows, columns=headers)

#print(df.head()) # yas slay queen

In [ ]:
# Load shapefile data (left sidebar!!)
shapefile_path = "/content/drive/MyDrive/GT/bluebonnet!!/mine - april 2023/New Hampshire/cb_2022_33_bg_500k/cb_2022_33_bg_500k.shp"

shapefile = gpd.read_file(shapefile_path)

shapefile.drop(shapefile.columns[[4, 7, 8]], axis=1, inplace=True)

shapefile['BOTH'] = shapefile['STATEFP'] + shapefile['COUNTYFP']

json = shapefile.to_json()

df = df.astype(float)

In [ ]:
# Create a base map centered at New Hampshire
map_center = [43.9654, -71.4742]  # Latitude and longitude of New Hampshire
m = folium.Map(location=map_center, zoom_start=8)

In [ ]:
# Overlay the demographic data as a choropleth map
folium.Choropleth(
    geo_data=json, #this is supposed to be a json file
    name="choropleth",
    data=df, #this is supposed to be a dataframe
    #columns=["State", "Total", "White", "Hispanic", "Asian Indian", "Bangladeshi"],
    columns=["State", "Hispanic"],
    key_on="feature.properties.STATEFP",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Population"
).add_to(m)

folium.LayerControl().add_to(m)

# Display the map
#m.save("new_hampshire_demographics_map.html")

m